In [ ]:
import numpy as np
import scipy as sc
import matplotlib.pyplot as plt

## Data set

In [ ]:
### Data sets:
### 1: Two linearly separable classes
### 2: XOR data
### 3: Sklearn moons
### 4: Sklearn circles
NDataSet = 1

In [ ]:
N = 1000 # Amount of data

### Constructs the dataset
if   NDataSet == 1:
    n = N // 2
    ### Gaussian input values for both dimensions / Class 1 with an offset in the second one
    X0 = np.random.randn(n,2)
    Y0 = np.zeros(n)
    #
    offset = 7.0
    X1 = np.random.randn(n,2) + offset
    Y1 = np.ones(n)
    #
    X = np.vstack([X0,X1])
    Y = np.hstack([Y0,Y1])
elif NDataSet == 2:
    n = N // 4
    ### Uniform input values for both dimensions
    offset = 4.0
    X0A = np.hstack([np.random.randn(n,1) - offset, np.random.randn(n,1) + offset])
    Y0A = np.zeros(n)
    X0B = np.hstack([np.random.randn(n,1) + offset, np.random.randn(n,1) - offset])
    Y0B = np.zeros(n)
    X1A = np.hstack([np.random.randn(n,1) + offset, np.random.randn(n,1) + offset])
    Y1A = np.ones(n)
    X1B = np.hstack([np.random.randn(n,1) - offset, np.random.randn(n,1) - offset])
    Y1B = np.ones(n)
    X = np.vstack([X0A,X0B,X1A,X1B])
    Y = np.hstack([Y0A,Y0B,Y1A,Y1B])
elif NDataSet == 3:
    from sklearn.datasets import make_moons
    X, Y = make_moons(n_samples = N, noise = 0.05)
elif NDataSet == 4:
    from sklearn.datasets import make_circles
    X, Y = make_circles(n_samples = N, noise = 0.05, factor = 0.5)
else:
    ### In sklearn there are other artificial data sets (see make_classification, for example)
    print("NDataSet not implemented")
    exit()


In [ ]:
### Scatter plot with the data
idxYclass0 = (Y[:] == 0)
idxYclass1 = (Y[:] == 1)
#print(type(X[idxYclass0, 0]), type(X[idxYclass0, 1]))
#print(len(X[idxYclass0, 0]), len(X[idxYclass0, 1]))
#print(X[idxYclass0, 0], X[idxYclass0, 1])
plt.scatter(X[idxYclass0, 0], X[idxYclass0, 1], s=3, c='blue') #outer circle 
plt.scatter(X[idxYclass1, 0], X[idxYclass1, 1], s=3, c='red')  #inner circle 
plt.axis('auto')
plt.show()

In [ ]:
print(X)
minX0 = np.min(X[:,0])
maxX0 = np.max(X[:,0])
minX1 = np.min(X[:,1])
maxX1 = np.max(X[:,1])
print(minX0,maxX0,minX1,maxX1)

## Direct (hand-made) reasonable solution

In [ ]:
###
### In some cases we can construct directly a solution
###

### This function will plot the output surface
def plotNetOutputFDataset(X,Y,strF):
    
    ### Indexes of the examples in every class (WE ARE ASSUMING WE ONLY HAVE 2 CLASSES)
    idxYclass0 = (Y[:] == 0)
    idxYclass1 = (Y[:] == 1)
    ### Minimum and maximum values of the input data
    minX0 = np.min(X[:,0])
    maxX0 = np.max(X[:,0])
    minX1 = np.min(X[:,1])
    maxX1 = np.max(X[:,1])
    
    ### Predictions surface
    NMesh = 50
    _x0 = np.linspace(minX0-0.2, maxX0+0.2, NMesh)
    _x1 = np.linspace(minX1-0.2, maxX1+0.2, NMesh)
    _Y  = np.zeros([NMesh, NMesh])

    for i0, x0 in enumerate(_x0):
        for i1, x1 in enumerate(_x1):
            _Xtmp = np.array([[x0, x1]])
            _Y[i1, i0] = Y = eval(strF)(_Xtmp)

    fig = plt.figure(constrained_layout=True, figsize=(5,5))
    ax  = plt.axes()
    #
    ax.pcolormesh(_x0, _x1, _Y, cmap='coolwarm', shading='auto')
    ax.axis('equal')
    ax.scatter(X[idxYclass0, 0], X[idxYclass0, 1], s=3, c='skyblue')
    ax.scatter(X[idxYclass1, 0], X[idxYclass1, 1], s=3, c='salmon')
    #
    return

###

def logisticFun(x):
    return 1.0 / (1.0 + np.exp(-x))

def fDataSet1(x):       ### For this data set we only need one (output) unit
    W = np.array([1,1])
    b = -7.0
    netInput = np.dot(x,W) + b
    netOutput = logisticFun(netInput)
    return netOutput

def fDataSet2(x):       ### For this data set we only need two hidden units and one output unit
    W11 = np.array([+0.3,+0.3])
    b11 = +0.5
    W12 = np.array([+0.3,+0.3])
    b12 = -0.5
    W21 = np.array([-1,+1])
    b21 = +0.
    #
    netInput11 = np.dot(x,W11) + b11
    netOutput11 = logisticFun(netInput11)
    netInput12 = np.dot(x,W12) + b12
    netOutput12 = logisticFun(netInput12)
    #
    x2 = np.array([netOutput11,netOutput12]).T
    netInput21 = np.dot(x2,W21) + b21
    netOutput21 = logisticFun(netInput21)
    #
    return netOutput21    

### Now call the proper function depending on the data set
if NDataSet == 1 or NDataSet == 2:
    if   NDataSet == 1:
        strF = "fDataSet1"
    elif NDataSet == 2:
        strF = "fDataSet2"
    #
    plotNetOutputFDataset(X,Y,strF)


## Trained solutions

In [ ]:
### Train step by step and plot of the training process
def trainModelSteps(neural_n,X,Y,nEpochs,plotEvery):

    import time
    from IPython.display import clear_output
    from ipywidgets.widgets.interaction import show_inline_matplotlib_plots

    ### Indexes of the examples in every class (WE ARE ASSUMING WE ONLY HAVE 2 CLASSES)
    idxYclass0 = (Y[:] == 0)
    idxYclass1 = (Y[:] == 1)
    ### Minimum and maximum values of the input data
    minX0 = np.min(X[:,0])
    maxX0 = np.max(X[:,0])
    minX1 = np.min(X[:,1])
    maxX1 = np.max(X[:,1])
    
    ### Initial fit (fit starts always from scratch and initializes all internal variables)
    neural_n.fit(X,Y)

    Accuracy = []
    for i in range(nEpochs):

        sX, sY = X, Y
        #Maybe we need this (https://stackoverflow.com/questions/24617356/sklearn-sgdclassifier-partial-fit)
        # to simulate shuffle=True (partial_fit does not shuffle?)
        #sR = [i for i in range(len(X))]
        #import random
        #random.shuffle(sR)
        #sX = [X[i] for i in sR]
        #sY = [Y[i] for i in sR]

        ### Partial fit (updates the model one iteration)
        neural_n.partial_fit(sX,sY,np.unique(sY))
        #print(neural_n.loss_curve_)

        ### Loss value (the loss_curve_ variable contains the whole history)
        Loss = neural_n.loss_curve_

        ### Accuracy
        current_accuracy = neural_n.score(X,Y)
        Accuracy.append(current_accuracy)

        if i % plotEvery ==0:

            ### Predictions surface
            NMesh = 50
            _x0 = np.linspace(minX0-0.2, maxX0+0.2, NMesh)
            _x1 = np.linspace(minX1-0.2, maxX1+0.2, NMesh)
            _Y  = np.zeros([NMesh, NMesh])

            for i0, x0 in enumerate(_x0):
                for i1, x1 in enumerate(_x1):
                    _Xtmp = np.array([[x0, x1]])
                    _Y[i1, i0] = neural_n.predict_proba(_Xtmp)[0][1]

            clear_output(wait=True)
            # Matplotlib - How to Change Subplot Sizes
            #  https://blog.finxter.com/matplotlib-how-to-change-subplot-sizes/
            #fig, ax = plt.subplots(1, 3, figsize=(15,5), gridspec_kw={'width_ratios': [2, 0.8, 0.8]})
            fig = plt.figure(constrained_layout=True, figsize=(10,10))
            ax = fig.add_gridspec(16, 16)
            ax0 = fig.add_subplot(ax[0:8,  0:16])
            ax1 = fig.add_subplot(ax[8:16, 0:8])
            ax2 = fig.add_subplot(ax[8:16, 8:16])
            #
            ax0.pcolormesh(_x0, _x1, _Y, cmap='coolwarm', shading='auto')
            ax0.axis('equal')
            ax0.scatter(X[idxYclass0, 0], X[idxYclass0, 1], s=3, c='skyblue')
            ax0.scatter(X[idxYclass1, 0], X[idxYclass1, 1], s=3, c='salmon')
            #
            ax1.plot(range(len(Loss)), Loss)
            ax1.set_xlabel('Epochs')
            ax1.set_ylabel('Loss')
            #
            ax2.plot(range(len(Accuracy)), Accuracy)
            ax2.set_xlabel('Epochs')
            ax2.set_ylabel('Accuracy')
            #
            plt.show()
            show_inline_matplotlib_plots()

            ## Figure without subplots...
            ##plt.pcolormesh(_x0, _x1, _Y, cmap='coolwarm', shading='auto')
            ##plt.axis('equal')
            ##plt.scatter(X[idxYclass0, 0], X[idxYclass0, 1], c='skyblue')
            ##plt.scatter(X[idxYclass1, 0], X[idxYclass1, 1], c='salmon')
            ##plt.show()
            ##
            ##plt.plot(range(len(Loss)), Loss)
            ##plt.xlabel('Epochs')
            ##plt.ylabel('Loss')
            ##plt.show()
            ##
            ##plt.plot(range(len(Accuracy)), Accuracy)
            ##plt.xlabel('Epochs')
            ##plt.ylabel('Accuracy')
            ##plt.show()

            time.sleep(0.5)

    print('Loss at the end of the training:',Loss[-1])
    print('Accuracy at the end of the training:',Accuracy[-1])
    return


In [ ]:
### Definition of the MLPClassifier (from sklearn)
#https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html

from sklearn.neural_network import MLPClassifier

HSizes     = ()         # Tuple: the ith element represents the number of neurons in the ith hidden layer
ActFun     = 'tanh'     # 'logistic' 'tanh' 'relu'
Solver     = 'sgd'      # 'sgd' 'adam'
LRate_init = 0.01       # 0.001
Momentum   = 0.9        # 0.9
LRate_schm = 'constant'
L2Reg      = 0.0        # 0.0001
EarlyStop  = False
MaxIterIni = 1          # We want to train epoch by epoch, so we perform an initial training of just 1 iteration
MaxEpochs  = 500        # This parameter should be the value of max_iter in a standard definition

neural_n = MLPClassifier(hidden_layer_sizes=HSizes, activation=ActFun, solver=Solver, alpha=L2Reg,\
                         learning_rate=LRate_schm, learning_rate_init=LRate_init, momentum=Momentum,\
                         early_stopping=EarlyStop, max_iter=MaxIterIni)

### General comments
### - 'adam' is usually faster than 'sdg' (specially for larger networks)
### - Very small learning rates (say, 0.00001) make the training process slow
### - Very large learning rates (say, 100.0)   make the training process degenerate
### - The momentum term accelerates the training process

###
### If NDataSet == 1, try
### - HSizes = ()    / ActFun = 'tanh' / Solver = 'sgd' / LRate_init = 0.001 / Momentum = 0.9 / MaxEpochs = 500
### - HSizes = (1)   / ActFun = 'tanh' / Solver = 'sgd' / LRate_init = 0.01  / Momentum = 0.9 / MaxEpochs = 500
###
### If NDataSet == 2, try
### - HSizes = (2)   / ActFun = 'tanh' / Solver = 'sgd' / LRate_init = 0.01 / Momentum = 0.9 / MaxEpochs = 500
### - HSizes = (5)   / ActFun = 'tanh' / Solver = 'sgd' / LRate_init = 0.01 / Momentum = 0.9 / MaxEpochs = 500
### - HSizes = (10)  / ActFun = 'tanh' / Solver = 'sgd' / LRate_init = 0.01 / Momentum = 0.9 / MaxEpochs = 500
### - HSizes = (50)  / ActFun = 'tanh' / Solver = 'sgd' / LRate_init = 0.01 / Momentum = 0.9 / MaxEpochs = 500
###   ... and you will see different solutions for the same problem
###
### If NDataSet == 3, try
### - HSizes = (20)  / ActFun = 'tanh' / Solver = 'sgd'  / LRate_init = 0.001 / Momentum = 0.9 / MaxEpochs = 500
### - HSizes = (20)  / ActFun = 'tanh' / Solver = 'sgd'  / LRate_init = 0.01  / Momentum = 0.9 / MaxEpochs = 500
### - HSizes = (20)  / ActFun = 'tanh' / Solver = 'sgd'  / LRate_init = 0.1   / Momentum = 0.9 / MaxEpochs = 500
###   ... to see the effect of the *learning rate*
### - HSizes = (20)  / ActFun = 'tanh' / Solver = 'sgd'  / LRate_init = 0.01  / Momentum = 0.9 / MaxEpochs = 500
### - HSizes = (20)  / ActFun = 'relu' / Solver = 'sgd'  / LRate_init = 0.01  / Momentum = 0.9 / MaxEpochs = 500
### - HSizes = (200) / ActFun = 'tanh' / Solver = 'sgd'  / LRate_init = 0.01  / Momentum = 0.9 / MaxEpochs = 500
### - HSizes = (200) / ActFun = 'tanh' / Solver = 'sgd'  / LRate_init = 0.01  / Momentum = 0.9 / MaxEpochs = 1000
###       (with ActFun = 'tanh', you will need "MaxEpochs = 1000")
### - HSizes = (200) / ActFun = 'relu' / Solver = 'sgd'  / LRate_init = 0.01  / Momentum = 0.9 / MaxEpochs = 500
###   ... to see the effect of the *activation function*
### - HSizes = (50)  / ActFun = 'tanh' / Solver = 'sgd'  / LRate_init = 0.01  / Momentum = 0.9 / MaxEpochs = 500
### - HSizes = (50)  / ActFun = 'tanh' / Solver = 'adam' / LRate_init = 0.01  / Momentum = 0.9 / MaxEpochs = 500
### - HSizes = (200) / ActFun = 'tanh' / Solver = 'sgd'  / LRate_init = 0.01  / Momentum = 0.9 / MaxEpochs = 500
### - HSizes = (200) / ActFun = 'tanh' / Solver = 'adam' / LRate_init = 0.01  / Momentum = 0.9 / MaxEpochs = 500
###   ... to see the effect of the optimization algorithm*
### - HSizes = (20)  / ActFun = 'tanh' / Solver = 'sgd'  / LRate_init = 0.01  / Momentum = 0.9 / MaxEpochs = 500
### - HSizes = (20)  / ActFun = 'relu' / Solver = 'adam' / LRate_init = 0.01  / Momentum = 0.9 / MaxEpochs = 500
###   ... are nice solutions
###
### If NDataSet == 4, try
### - The same configurations than for NDataSet = 3
### - You will see a similar behaviour, but less smooth in some cases


In [ ]:
### Train and plot
plotEvery = 25
trainModelSteps(neural_n,X,Y,MaxEpochs,plotEvery)